# Tutorial 6 - Import user created species into pyDEW
Following tutorial 5 where we created a coder species which would allow us to change parameters as we want, we can now import it and do calculations!

Import the required packages, including running the core notebook again. This code will be migrated into either ENKI or pyDEW soon.

In [1]:
import numpy as np
import dill
import pandas as pd
%run z_tutorial5_core.ipynb
from thermoengine import model, phases
from scipy.optimize import root_scalar, minimize
import pyDEW
from copy import copy

/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Load the species we generated before:

In [2]:
load_coder_modules('tutorial5_working')
with open('tutorial5_species.pkl','rb') as file:
    tut5 = dill.load(file)
    file.close()
print('Number of species = '+str(len(tut5)))

Number of species = 1


We now have a thermoengine phase object contained in a dictionary:

In [3]:
tut5

{'NACL(AQ)': <thermoengine.phases.PurePhase at 0x7fb776a1d910>}

## Set up the DEW system
Here's how we can add the new complex to the default system. See below for how to add it to a custom system:

In [4]:
other_species = copy(pyDEW.defaultsystem.other_species_names) # It is important to copy it, as otherwise you remove NACL(AQ) from the default list for the whole session. This is something I need to fix!
other_species.pop(other_species.index('NACL(AQ)')) # Remove the default NACL(AQ) complex! If it's a totally new species you don't need to do this.
other_species += [tut5['NACL(AQ)']]
sys = pyDEW.System(other_species=other_species)

You can then proceed with your calculations as normal!

To make a new system, we can just pass the new species object as part of the list:

In [5]:
elements = ['H','O','Na','Cl']
basis_species = ['H2O','H+','NA+','CL-','O2(G)']
other_species = ['OH-',tut5['NACL(AQ)']]
hydrated_species = pyDEW.defaultsystem.hydrated_species # If the new complex has extra H2O molecules, add it this dictionary using something like:
# hydrated_species['NACL(AQ)'] = 10
sys = pyDEW.System(elements = elements,
                   basis_species = basis_species,
                   other_species = other_species,
                   hydrated_species=hydrated_species)

## Change the species' parameters
Maybe you want to change the parameters again. What can we change?

In [6]:
tut5['NACL(AQ)'].param_names

['T_r',
 'P_r',
 'G_ref',
 'S_ref',
 'v_ref',
 'C_p_ref',
 'Psi',
 'eta',
 'rH',
 'theta',
 'z',
 'complex',
 'gas',
 'prd_ac',
 'prd_omega',
 'a1',
 'a2',
 'a3',
 'a4',
 'c1',
 'c2',
 'omega0']

To make things easy, we will save this list to a variable so that we can quickly look up the position of whatever parameter we want to change:

In [7]:
param_ids = tut5['NACL(AQ)'].param_names

We can calculate the logK at an abitrary temperature and pressure to see the effects of our changes:

In [8]:
sys.logK(tut5['NACL(AQ)'], t = 500.0, p = 10000.0)

To change the reference free energy and entropy (to ridiculous values):

In [9]:
tut5['NACL(AQ)'].set_param_values(np.array([param_ids.index('G_ref'), param_ids.index('S_ref')]),
                                  np.array([-1000.0, 10.0])
                                 )

How has the logK changed?

In [10]:
sys.logK(tut5['NACL(AQ)'], t = 500.0, p = 10000.0)

To set some of the parameters manually, e.g., $\omega_0$, you have to change the switch and the value together:

In [11]:
tut5['NACL(AQ)'].set_param_values(np.array([param_ids.index('prd_omega'), param_ids.index('omega0')]),
                                  np.array([0, 500])
                                 )

And the logK again?

In [12]:
sys.logK(tut5['NACL(AQ)'], t = 500.0, p = 10000.0)